In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import os

container_beam = os.path.expanduser("~/beam_root")
root = container_beam if os.path.isdir(container_beam) else "../.."
root

'/home/jovyan/beam_root'

In [8]:
file_name = "Daycab_new_400kW-speed_mph&grade_percent&mass_kg_lookup.csv"
rates = pd.read_csv(f"{root}/data_files/route_e/{file_name}")
# flat_surface = rates[(rates['grade'] == 0)]
# display(flat_surface)
# sns.lineplot(data=flat_surface, x="speed", y="energy_rate")
rates

,speed,grade,mass_kg,energy_rate
0,1,-20.0,11337.86848,0.417743
1,1,-20.0,11799.78164,0.417743
2,1,-20.0,12261.69480,0.417743
3,1,-20.0,12723.60796,0.417743
4,1,-20.0,13185.52112,0.417743
...,...,...,...,...
445495,100,20.0,34433.52650,0.301870
445496,100,20.0,34895.43966,0.299282
445497,100,20.0,35357.35282,0.299282
445498,100,20.0,35819.26598,0.299282


In [9]:
def create_range(column, border, initial_border, group):
    border_column = "low_" + column if border == 'low' else "high_" + column
    shft = 1 if border == 'low' else -1
    rates[border_column] = rates.groupby(group)[column].shift(shft)
    rates[border_column] = rates[column] + (rates[border_column] - rates[column]) / 2
    rates[border_column].replace(to_replace=np.nan, value=initial_border, inplace=True)


create_range('speed', "low", 0, ['grade', 'mass_kg'])
create_range('speed', "high", 120, ['grade', 'mass_kg'])
create_range('mass_kg', "low", 10000, ['grade', 'speed'])
create_range('mass_kg', "high", 50000, ['grade', 'speed'])
create_range('grade', "low", -25, ['mass_kg', 'speed'])
create_range('grade', "high", 25, ['mass_kg', 'speed'])
rates[rates['grade'] == 20]

,speed,grade,mass_kg,energy_rate,low_speed,high_speed,low_mass_kg,high_mass_kg,low_grade,high_grade
440000,1,20.0,11337.86848,1.319509,0.0,1.5,10000.00000,11568.82506,19.75,25.0
440001,1,20.0,11799.78164,1.319509,0.0,1.5,11568.82506,12030.73822,19.75,25.0
440002,1,20.0,12261.69480,1.319509,0.0,1.5,12030.73822,12492.65138,19.75,25.0
440003,1,20.0,12723.60796,1.316733,0.0,1.5,12492.65138,12954.56454,19.75,25.0
440004,1,20.0,13185.52112,1.316733,0.0,1.5,12954.56454,13416.47770,19.75,25.0
...,...,...,...,...,...,...,...,...,...,...
445495,100,20.0,34433.52650,0.301870,99.5,120.0,34202.56992,34664.48308,19.75,25.0
445496,100,20.0,34895.43966,0.299282,99.5,120.0,34664.48308,35126.39624,19.75,25.0
445497,100,20.0,35357.35282,0.299282,99.5,120.0,35126.39624,35588.30940,19.75,25.0
445498,100,20.0,35819.26598,0.299282,99.5,120.0,35588.30940,36050.22256,19.75,25.0


In [10]:
rates["speed_mph_float_bins"] = rates.apply(lambda row: f"[{row['low_speed']}, {row['high_speed']})", axis=1)
rates["grade_percent_float_bins"] = rates.apply(lambda row: f"[{row['low_grade']}, {row['high_grade']})", axis=1)
rates["mass_kg_float_bins"] = rates.apply(lambda row: f"[{row['low_mass_kg']}, {row['high_mass_kg']})", axis=1)
rates["rate"] = rates['energy_rate'] * 100
rates

,speed,grade,mass_kg,energy_rate,low_speed,high_speed,low_mass_kg,high_mass_kg,low_grade,high_grade,speed_mph_float_bins,grade_percent_float_bins,mass_kg_float_bins,rate
0,1,-20.0,11337.86848,0.417743,0.0,1.5,10000.00000,11568.82506,-25.00,-19.75,"[0.0, 1.5)","[-25.0, -19.75)","[10000.0, 11568.82506)",41.774312
1,1,-20.0,11799.78164,0.417743,0.0,1.5,11568.82506,12030.73822,-25.00,-19.75,"[0.0, 1.5)","[-25.0, -19.75)","[11568.82506, 12030.73822)",41.774312
2,1,-20.0,12261.69480,0.417743,0.0,1.5,12030.73822,12492.65138,-25.00,-19.75,"[0.0, 1.5)","[-25.0, -19.75)","[12030.73822, 12492.65138)",41.774312
3,1,-20.0,12723.60796,0.417743,0.0,1.5,12492.65138,12954.56454,-25.00,-19.75,"[0.0, 1.5)","[-25.0, -19.75)","[12492.65138, 12954.56454)",41.774312
4,1,-20.0,13185.52112,0.417743,0.0,1.5,12954.56454,13416.47770,-25.00,-19.75,"[0.0, 1.5)","[-25.0, -19.75)","[12954.56454, 13416.4777)",41.774312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445495,100,20.0,34433.52650,0.301870,99.5,120.0,34202.56992,34664.48308,19.75,25.00,"[99.5, 120.0)","[19.75, 25.0)","[34202.56992, 34664.48308)",30.186988
445496,100,20.0,34895.43966,0.299282,99.5,120.0,34664.48308,35126.39624,19.75,25.00,"[99.5, 120.0)","[19.75, 25.0)","[34664.48308, 35126.39624)",29.928165
445497,100,20.0,35357.35282,0.299282,99.5,120.0,35126.39624,35588.30940,19.75,25.00,"[99.5, 120.0)","[19.75, 25.0)","[35126.39624, 35588.3094)",29.928165
445498,100,20.0,35819.26598,0.299282,99.5,120.0,35588.30940,36050.22256,19.75,25.00,"[99.5, 120.0)","[19.75, 25.0)","[35588.3094, 36050.222559999995)",29.928165


In [11]:
rates[['speed_mph_float_bins', 'grade_percent_float_bins', 'mass_kg_float_bins', 'rate']]\
    .to_csv(f"{root}/test/input/beamville/freight/{file_name}", index=False)
